# connie-skipper

Repository at [https://github.com/PhMota/connie-skipper](https://github.com/PhMota/connie-skipper)

In [1]:
%load_ext autoreload
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
# import gc
from glob import glob
import xarray as xr
import connie_skipper as sk
import traceback
import re, os, time
import warnings
warnings.simplefilter("ignore", category=xr.core.extensions.AccessorRegistrationWarning)

In [2]:
from IPython.display import display, IFrame, clear_output
from ipywidgets import interact, interactive_output
import ipywidgets as widgets

In [3]:
sortpattern = lambda p: int(re.search(r"([0-9]+)\.fz", p).groups()[0])
class App:
    def select_update(self, pattern):
        size_fmt = lambda file: str(int(os.stat(file).st_size/1e6))
        path_fmt = lambda file: file.split(pattern.split("*")[0])[-1]
        time_fmt = lambda file: time.strftime( "%Y-%m-%d %H:%M", time.localtime(os.stat(file).st_ctime) )  #time.strftime( , 
        fmt = lambda file: f'{time_fmt(file)} {path_fmt(file)} [{size_fmt(file)}M]'
        files = sorted( glob(pattern), key=time_fmt )[::-1] 
        self.select.options = [(".", None)] + [ ( fmt(file), file ) for file in files ]
        
    def __init__(self, rootpath):
        self.da = None
        
        self.output = widgets.Output()
        self.pattern = widgets.Text(
            value = rootpath,
            placeholder = 'pattern for files',
            description = "file pattern",
            layout = { "width": "90%" }
        )
        self.select = widgets.Select( 
            options = [],
            layout = {'width': '90%'},
            rows = 10,
        )
        self.select_update(self.pattern.value)
        self.pattern.on_submit(
            lambda change: self.select_update(self.pattern.value)
        )
        self.progress = widgets.IntProgress(
            value = 0, 
            min = 0, 
            max = 10, 
            description = 'loading'
        )
        self.progressPlot = widgets.IntProgress(
            value = 0, 
            min = 0, 
            max = 10, 
            description = 'plotting'
        )
        self.app = None
        with self.output:
            display(self.pattern, self.select, self.progress)
        display(self.output)
        self.select.observe( 
            lambda path: self.plot_from_path2(path.new) if path.new is not None else None,
            names = "value"
        )
        
        self.interactive_figure = None
        trim = widgets.Checkbox(
            description = 'trim', 
            value = True
        )
        demod = widgets.Checkbox(
            description = 'demod/osi', 
            value = True
        )
        statsProj = widgets.Dropdown( 
            options = ["median", "mean", "std", "mad", "min", "max"], 
            description = 'stats(proj)' 
        )
        median_row = widgets.Checkbox(
            description = 'median(row)',
            value = True
        )
        median_col = widgets.Checkbox(description='median(col)')
        spectrum = widgets.Checkbox(description='spectrum', value=True)
        chid = widgets.IntSlider(
            value = 0,
            min = 0,
            max = 1,
            description = f'chid',
            continuous_update = False
        )
        samp = widgets.IntSlider( 
            value = 0,
            min = 0, 
            max = 1,
            description = f'samp',
            continuous_update = False
        )
        stats = widgets.Dropdown( options=["none", "median", "mean", "std", "mad", "min", "max"], description='stats(samp)' )
        stats.value = "mean"
        statsRange = widgets.IntRangeSlider(
            value = [0,1],
            min = 0,
            max = 1,
            description = f"{stats.value}",
            continuous_update = False
        )
        energyFactor = widgets.IntSlider(
            value = 1,
            min = 1,
            max = 49,
            description = "percentile",
            continuous_update = False
        )
        threshold = widgets.FloatSlider(
            description = 'threshold', 
            value = 0,
            max = 100,
            min = 0,
            continuous_update = False,
        )        
        self.path = widgets.Label(
            value = ""
        )
        self.log = widgets.HTML(
            value = "",
            description = "messages",
        )
#         stats.observe( lambda opt: setattr(statsRange, "description", f"{opt.new}(samp)"), names="value" )
        ui = widgets.VBox([
            self.path,
            chid, 
            energyFactor,
            samp, 
            stats, 
            statsRange,
            trim, 
            demod, 
            statsProj, 
            median_row, 
            median_col, 
            spectrum, 
            threshold,
            self.progressPlot,
        ])
        self.app = {
            "path": self.path,
            "median_row": median_row, 
            "median_col": median_col, 
            "spectrum": spectrum,
            "trim": trim, 
            "demod": demod, 
            "statsProj": statsProj, 
            "chid": chid, 
            "samp": samp, 
            "stats": stats, 
            "stats_range": statsRange,
            "energy_percentile": energyFactor,
            "threshold": threshold,
        }
        self.plot = interactive_output( 
            self.gen_plot, 
            self.app
        )
        with self.output:
            display( widgets.HBox([ui, self.plot]) )
            display( self.log )
        
        
    def plot_from_path2(self, path):

        self.log.value = ""
        self.progress.value, self.progress.description = 1, "opening file"
        self.da = sk.fits_to_DataArray( path )
        self.log.value += self.da._repr_html_()
        
        self.progress.value, self.progress.description = 5, "widgets"
        chid_max = len(self.da["chid"].data)-1
        samp_max = int(self.da["samp"].data[-1])

        widget = self.app["stats_range"]
        _ = widget._trait_notifiers
        widget._trait_notifiers = {}
        widget.min = 0
        widget.max = samp_max+1
        widget.value = [0 if widget.max <= 100 else 10, widget.max]
        widget._trait_notifiers = _
        
        widget = self.app["chid"]
        _ = widget._trait_notifiers
        widget._trait_notifiers = {}
        widget.max = chid_max
        widget.value = 2
        widget.description = f'chid[0─{chid_max}]'
        widget._trait_notifiers = _

        widget = self.app["samp"]
        _ = widget._trait_notifiers
        widget._trait_notifiers = {}
        widget.max = samp_max
        widget.description = f'samp[0─{samp_max}]'
        widget._trait_notifiers = _

        widget = self.app["threshold"]
        _ = widget._trait_notifiers
        widget._trait_notifiers = {}
        widget.max = np.max(self.da)
        widget._trait_notifiers = _

        self.path.value = path.split('/')[-1]
#         self.log.value += f"{dir(self.plot)}<br>"
#         self.log.value += f"{self.plot.notify_change.__doc__}<br>"
#         self.plot.notify_change()

#         self.gen_plot(**self.app)
#         self.app["chid"].value = 2

        self.progress.value, self.progress.description = 10, "complete"
        
    def gen_plot(self, 
        path,
        median_row, 
        median_col, 
        spectrum,
        demod,
        trim, 
        chid, 
        samp, 
        stats, 
        stats_range,
        statsProj,
        energy_percentile,
        threshold,
    ):
        if self.da is None:
            self.log.value += f"nothing to plot<br>"
            return
        self.progressPlot.value = 1
        self.log.value += f"'{self.da.attrs['path'][0]}': {{<br>"
#         self.log.value += f"{self.da.attrs['name']}<br>"
#         self.log.value += ''.join( [ f"<b>{key}</b> {value}<br>" for key, value in self.da.attrs.items() ] )
        try:
            op = lambda _da: _da.skipper.isel(chid=chid)
            if stats == "none":
                stats_str = stats
                op = lambda _da, prevop=op: prevop(_da).skipper.isel(samp=samp)
            else:
                stats_str = f"{stats}({stats_range[0]}, {stats_range[1]})"
                op = lambda _da, prevop=op: prevop(_da).skipper.isel(samp=slice(*stats_range)).skipper.stats(dim = "samp", mode = stats)
            if trim:
                op = lambda _da, prevop=op: prevop(_da).skipper.trim()
            if demod:
                op = lambda _da, prevop=op: prevop(_da).skipper.demodulate("col")
            else:
                op = lambda _da, prevop=op: prevop(_da).skipper.centralize_os("col")
            if threshold > 0:
                op = lambda _da, prevop=op: prevop(_da).skipper.extract(threshold, nborder=0, log=self.log)
                spectrum = False
                median_row = False
                median_col = False
            suptitle = self.da.attrs['path'][0].replace("_", r"\_")
            self.interactive_figure = op(self.da).skipper.plot_full( 
                robust = True, 
                fig = self.interactive_figure, 
#                 title = ,
                xproj = median_row,
                yproj = median_col,
                spectrum = spectrum,
                energy_percentile = energy_percentile,
                mode = statsProj,
                suptitle = fr"{suptitle}\\chid={chid} samp={samp} stats={stats_str}",
                progress_bar = (self.progressPlot, 0, 10),
                log = self.log
            )
        except:
            self.log.value += "<b style='color:red'>error in plot</b><br>"
            self.log.value += traceback.format_exception()
        finally:
#             self.log.value += f"{self.da.attrs['name']}<br>"
            self.log.value += f"}},<br>"

"/share/storage2/connie/data/lta/testAngra/"
App("/share/storage2/connie/data/runs/101/*NSAMP400*.fz");

Output()

# Noise & DC evolution

In [27]:
temperatures = {
    (0,127): '90K',
    (127,205): '80K',
    (205, 286): '70K',
    (286, None): '60K'
#     (None, 0): '120K',
}

evo = {
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img344.fz': {
        'os': [0.27147672996830297, 59.145641740126365, 88069.66832732815],
        'oserr': [0.1046640395494369, 0.0755022940433873, 154.8508005531048],
        'ac': [-10.12913832891893, 64.3380607050696, 395.6482532466433, 0.006024378686630945, 195626.12344619498],
        'acerr': [0.16428554018597907, 0.12077237115961732, 2.202826913427452, 0.00019842074425636627, 484.18462683992226],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img347.fz': {
        'os': [-0.15597828109980305, 59.348556657832496, 88090.26376277758],
        'oserr': [0.07627971807390886, 0.054848659945606414, 112.60059134202162],
        'ac': [-10.892695966921657, 64.22348596591961, 393.34217406361415, 0.005786377083881077, 195390.15465487604],
        'acerr': [0.1299867917334663, 0.09554645874764502, 1.844128376563742, 0.00015422238116148516, 384.08965865447203],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img350.fz': {
        'os': [-0.448734547836224, 59.13356840460914, 87984.98682112915],
        'oserr': [0.07546543224373406, 0.05415124424226385, 111.62332901432768],
        'ac': [-11.30705058289569, 64.32975667461693, 388.4458741748635, 0.005782892279309043, 195332.35078518276],
        'acerr': [0.12218758813025317, 0.08998293124092926, 1.7121495209245916, 0.00014492579261960305, 360.3775839181744],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img353.fz': {
        'os': [-0.17617549945314703, 59.18849716681866, 88105.74697617478],
        'oserr': [0.0923615286789604, 0.06652020775968254, 136.66546270848065],
        'ac': [-10.53060914268363, 64.30032293027027, 393.5790109279429, 0.005337272928472473, 195606.19851521202],
        'acerr': [0.15317708844392605, 0.11298344656086048, 2.262046451124507, 0.0001751861430699349, 453.39520398648614],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img356.fz': {
        'os': [-0.28307343246545114, 59.194064039232565, 87994.04288859108],
        'oserr': [0.1768238329447466, 0.12652594560503252, 261.2374030417729],
        'ac': [-10.582233967659871, 64.26300556592814, 399.44103200005264, 0.005924130996140085, 196170.35564070428],
        'acerr': [0.24473715098373525, 0.18070871396609725, 4.972539018801344, 0.00035470441634164373, 727.7618366552333],
    },
#     '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img362.fz': {
#         'os': [-0.36741055835481673, 59.83616678351553, 88373.3905336818],
#         'oserr': [0.029736104284830527, 0.021540815194944912, 43.68781985487168],
#         'ac': [-10.836747885089219, 65.16617981662836, 192105.6432917677],
#         'acerr': [0.1606487200968165, 0.12018861886387909, 462.3944598052983],
#     },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img365.fz': {
        'os': [-0.17260131243868485, 59.079263826087946, 87982.45783256533],
        'oserr': [0.18554821864751644, 0.132771952329175, 274.7535356598364],
        'ac': [-11.115790418890471, 64.25554185332628, 392.9764878093885, 0.005250370369858846, 196677.37903861306],
        'acerr': [0.25244393125236303, 0.18643980990664055, 12.068136554985214, 0.0009383129845327973, 772.8060542754589],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img368.fz': {
        'os': [-0.03336715105609905, 59.08883919146027, 88020.98786319465],
        'oserr': [0.08265984611599456, 0.059580152604506165, 122.44757423707367],
        'ac': [-10.273107041850594, 64.35901191079287, 390.5579778136876, 0.005380735936457886, 195293.25552092947],
        'acerr': [0.1395105755409875, 0.10245734416993497, 2.042523381191709, 0.00015976161983969112, 411.2367165114132],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img371.fz': {
        'os': [-0.09639361466238945, 59.841394019135095, 88009.5766108614],
        'oserr': [0.11852312483007152, 0.0855414163533043, 173.1289563586379],
        'ac': [-9.853486865866367, 64.69359856103787, 390.1925940572807, 0.005076458157148605, 195799.36317542932],
        'acerr': [0.17380848607048724, 0.12794685860627686, 2.662928053502271, 0.0001927284833655809, 510.5285541730195],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img374.fz': {
        'os': [0.14366833417481786, 60.4488066831905, 88391.62139786377],
        'oserr': [0.028460037703527653, 0.020535539742362593, 41.36472411652955],
        'ac': [-9.911875599729319, 65.36846515864211, 388.9547429851894, 0.006941153717373939, 195008.74868954718],
        'acerr': [0.11222284376361821, 0.0831921017884977, 1.416927376435684, 0.0001415099094985924, 324.9189370174893],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img265.fz': {
        'os': [-0.03183299294523615, 59.44437882484635, 88379.5868321552],
        'oserr': [0.031006324777115454, 0.022437817769775055, 45.87690858943826],
        'ac': [-10.515996645694441, 64.85540861182564, 393.7464906854457, 0.02414050695600855, 193985.56281434576],
        'acerr': [0.13125639265245612, 0.09689370849530596, 0.8574587353463816, 0.00031125416811244325, 378.0576920940795],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img262.fz': {
        'os': [0.004270473788954843, 58.80636766847501, 88033.39493716498],
        'oserr': [0.11469617548534922, 0.0829551739753356, 170.74326394075825],
        'ac': [-10.158539136234115, 63.936057015508844, 390.67473922565426, 0.022037799583555954, 195619.91307131856],
        'acerr': [0.1748976959658906, 0.1280141649721186, 1.163535012653676, 0.0004027662110675148, 515.2295107039223],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img259.fz': {
        'os': [0.004897015506633436, 59.14722521704246, 88042.20828660316],
        'oserr': [0.1667651813079158, 0.12064253837346801, 246.71329382483475],
        'ac': [-10.885742748660112, 64.8196914702082, 399.8976280868877, 0.022534528095361217, 196792.11542365872],
        'acerr': [0.23170916653626014, 0.1726456442165719, 2.056790385519466, 0.0005887325086663053, 679.2709988629975],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img226.fz': {
        'os': [-0.05467111918895371, 59.111558820673366, 87878.72685541325],
        'oserr': [0.17920507566295985, 0.12954066422347865, 264.8871669972684],
        'ac': [-10.66825611312577, 64.593737638463, 397.19361379413806, 0.03109948350693677, 196348.24438430744],
        'acerr': [0.23757503494184878, 0.1745921149488626, 1.4826397583200788, 0.0006574839681989369, 692.5469147922156],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img223.fz': {
        'os': [-0.36888835607561693, 59.86874815274703, 88373.89264970833],
        'oserr': [0.031242703352735934, 0.02266313974160283, 45.87977002033806],
        'ac': [-11.456254119913734, 65.06289622754333, 396.71036644440557, 0.03656564800068028, 194484.31185894896],
        'acerr': [0.12454602469674692, 0.09134548365411939, 0.6656548613907574, 0.00036414154695845244, 356.40256694960675],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img220.fz': {
        'os': [-0.06399802600177462, 59.22554875754211, 88180.30589274096],
        'oserr': [0.05046357250489105, 0.03634338571682551, 74.72157736363253],
        'ac': [-10.348503809481981, 64.39293359147872, 392.21621957694316, 0.03684167479650516, 195024.77093609914],
        'acerr': [0.11216581791112006, 0.08232712887221395, 0.6029290755001332, 0.0003341259686304618, 325.3242449792411],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img217.fz': {
        'os': [-0.3863382645942123, 59.58911256042837, 88292.94264567898],
        'oserr': [0.03907193269296789, 0.028497028183794765, 57.57816080537777],
        'ac': [-10.680944362263082, 65.29035882331054, 399.3752230367917, 0.0421317989172419, 194287.51233580417],
        'acerr': [0.13143142469407096, 0.09645801247135083, 0.6494938266570955, 0.0004114767740885267, 373.03571627162444],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img214.fz': {
        'os': [-0.044306622991388274, 59.76667946666859, 88335.05314335844],
        'oserr': [0.03635885303466363, 0.02623072818015164, 53.46739414069419],
        'ac': [-11.376774965077942, 64.96430482265082, 396.37059780419946, 0.046479644615144956, 194550.70982219666],
        'acerr': [0.12521801286917034, 0.09186583344060807, 0.5910647283070614, 0.0004152369121496606, 357.3689713498255],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img211.fz': {
        'os': [-0.3311437196170958, 60.07854621008767, 88314.65348710123],
        'oserr': [0.03832347049680452, 0.02782539042364286, 56.008492340262734],
        'ac': [-10.424032223052352, 65.0719007148932, 399.8096672709568, 0.050250769561098224, 194719.93726274738],
        'acerr': [0.12325783518269542, 0.09030280003584394, 0.5599146601697115, 0.0004248586027000665, 350.8849722615305],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img208.fz': {
        'os': [0.025819529521454533, 59.826429271910385, 88000.11508157423],
        'oserr': [0.09901666095864133, 0.07131476057324215, 144.70199851339817],
        'ac': [-10.909839573631322, 64.73411794453345, 398.63071300309053, 0.05384631988760957, 195362.11161094953],
        'acerr': [0.1695477902786059, 0.12262241721845331, 0.727053151411895, 0.0006076756559966188, 484.18653182005175],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img205.fz': {
        'os': [-0.16001021300658633, 59.87975194101189, 87851.6638617194],
        'oserr': [0.15539052349041751, 0.11179152576789449, 226.5606823082084],
        'ac': [-10.641883724384876, 65.55349633378125, 396.34928256250913, 0.05577501727915416, 196156.3789600185],
        'acerr': [0.21643211056469433, 0.15720859941773543, 0.9226214760744849, 0.0007807029274634371, 612.3044379232293],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img202.fz': {
        'os': [-0.23532926817021818, 60.117688683480814, 88211.86237647118],
        'oserr': [0.048313902891916324, 0.035003976410455315, 70.4800000001484],
        'ac': [-11.139308622194472, 65.86894323081486, 398.29460129263066, 0.05664621037042846, 195148.01233105652],
        'acerr': [0.11972858592862011, 0.08743061349209567, 0.5118575203565269, 0.00043338412349550276, 335.7172249548124],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img199.fz': {
        'os': [-0.12311983644055471, 60.05650327914276, 88051.82082231266],
        'oserr': [0.06091732537905832, 0.043960853650065294, 88.78817174772232],
        'ac': [-10.169014946177283, 65.2357483827198, 396.8852460676327, 0.05604883195748327, 194955.44539413083],
        'acerr': [0.12034048034997807, 0.08725541244875464, 0.5068002681421382, 0.00043772244931685193, 340.5080315178013],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img196.fz': {
        'os': [-0.05258515381098764, 59.41236469486758, 88205.32382475262],
        'oserr': [0.049945561655725244, 0.03597272103037653, 73.7604443251314],
        'ac': [-10.805876229213329, 64.87711949990853, 399.8866989481744, 0.05750997731472739, 194753.1642715616],
        'acerr': [0.12351689308112396, 0.08946718678178417, 0.5215603793833078, 0.00045772824303003194, 350.9894893245123],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img193.fz': {
        'os': [-0.2653396792178026, 59.07250597036815, 87994.60229330168],
        'oserr': [0.06611348201959788, 0.04756115672740504, 97.85217727968521],
        'ac': [-11.061433732249448, 64.53978267742569, 397.68268661126154, 0.057904326669296846, 195143.39789889802],
        'acerr': [0.11271998700100082, 0.08190550469360043, 0.46288934361686357, 0.0004048847902423439, 322.5866089835207],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img190.fz': {
        'os': [0.03119834089240766, 59.02208919543296, 87921.42281537504],
        'oserr': [0.09093605533092722, 0.06577548987198077, 134.72445589352012],
        'ac': [-10.791740366320717, 64.50521907107216, 394.66027119734224, 0.05595559143792907, 195263.09279017575],
        'acerr': [0.15549399027623537, 0.11212672701924992, 0.6590490076345296, 0.0005718555586942488, 445.4862080436758],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img187.fz': {
        'os': [0.18985357393635974, 58.94921570827856, 87854.67493715907],
        'oserr': [0.09318099709237783, 0.06681724893542192, 138.2968274462305],
        'ac': [-10.625025539567332, 64.63039647598875, 396.19770630131325, 0.0589322328053542, 194944.45949823858],
        'acerr': [0.15092263781566415, 0.10792179511246679, 0.6151961665094916, 0.0005480753976230082, 431.88444588635946],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img184.fz': {
        'os': [-0.3650365244138931, 59.139675194066825, 87880.53053078272],
        'oserr': [0.149612931342704, 0.10712883343844723, 221.11526213643663],
        'ac': [-11.306578649790195, 64.5931235951188, 396.1744590812212, 0.057412940148017795, 195781.87385586742],
        'acerr': [0.2280144028689799, 0.16554456178200008, 0.9574343754466229, 0.0008498399623359041, 655.1325982970418],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img181.fz': {
        'os': [-0.2947009701239745, 59.489449488481355, 87876.88868934374],
        'oserr': [0.08720334598201776, 0.06289360545579782, 128.02071558328856],
        'ac': [-10.79294350351032, 64.64889188622685, 400.10708483933723, 0.058429983358812734, 195223.3936875028],
        'acerr': [0.1337858103499695, 0.09716594898677822, 0.5459229349041237, 0.00048172565126986395, 382.1670846436872],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img172.fz': {
        'os': [-0.5080138380012229, 59.73979229247933, 88280.33043211675],
        'oserr': [0.04116326425985355, 0.02990437864547487, 60.50929596124626],
        'ac': [-11.08621036452469, 65.05127097447632, 399.31450637432954, 0.059279021296615914, 194759.7899844511],
        'acerr': [0.12400566526014921, 0.09027183559827984, 0.5194678626248276, 0.0004664974458035732, 351.83897662366775],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img175.fz': {
        'os': [-0.12318985142775424, 59.6168900481668, 88043.70534711864],
        'oserr': [0.059869139894729084, 0.04295589070340928, 87.95480457723501],
        'ac': [-11.513545910725831, 65.01090593408946, 401.74290565250794, 0.058747231401083976, 195104.37010938284],
        'acerr': [0.11683673242964468, 0.08465667208715605, 0.4886995724965246, 0.0004377349776392954, 332.25460002976945],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img178.fz': {
        'os': [-0.09440888886118483, 59.29045329455836, 88009.55244719776],
        'oserr': [0.1724025283795448, 0.1235015582284527, 254.4573451104596],
        'ac': [-11.040277882118023, 65.24232117185493, 401.1111480258752, 0.057424620701946136, 196961.9946182767],
        'acerr': [0.2422194514918753, 0.1789919539227636, 1.2042593136773057, 0.0009277073981869929, 694.2162025696215]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img169.fz': {
        'os': [-0.20859081124243195, 59.235486757979864, 87861.28811193097],
        'oserr': [0.13668607098827856, 0.09900597141194142, 201.69864428856928],
        'ac': [-11.153530727171127, 65.20187682097273, 397.63122187993355, 0.059704829522168705, 195685.73310275516],
        'acerr': [0.1950924705883128, 0.141420443197749, 0.7860025414500738, 0.0007043108373317838, 553.1052394932109],  
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img166.fz': {
        'os': [-0.020175608858074307, 59.59943382552798, 87980.82580917062],
        'oserr': [0.06886388773660937, 0.04960142310011065, 101.00515205203381],
        'ac': [-10.521841586281463, 64.96438359102828, 399.52865393650154, 0.05901887142733882, 194963.21578329353],
        'acerr': [0.12536650169770894, 0.09107895692161581, 0.5213665660745976, 0.000470663333955102, 355.8505636471652]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img163.fz': {
        'os': [-0.27603895527298167, 60.28720015638354, 88881.8621480748],
        'oserr': [0.054275802986569276, 0.0395327175871201, 79.53603171315038],
        'ac': [-11.070384811532074, 65.33998874528783, 399.21378440491975, 0.061283403820100814, 194037.09368120442],
        'acerr': [0.14405544941206994, 0.10495556696023603, 0.5923821797578306, 0.0005476689836020455, 404.03076649358667]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img160.fz': {
        'os': [-0.2537135990658845, 59.61654839442997, 88126.87687673673],
        'oserr': [0.05602953839251744, 0.040218203106513285, 82.37103877671717],
        'ac': [-11.098604430044892, 65.25990584080998, 398.28341878907287, 0.06022856518187983, 195050.39459906166],
        'acerr': [0.1193748082811478, 0.08644792398509188, 0.49395217768263305, 0.0004509860162141366, 337.32870389897647],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img157.fz': {
        'os': [ -0.018, 59.636, 87564.396],
        'ac': [ -10.59, 65.059, 399.233, 0.059, 195685.933]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img154.fz': {
        'os': [ -0.172, 59.655, 87554.441],
        'ac': [ -11.456, 65.035, 401.517, 0.065, 194096.201]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img151.fz': {
        'os': [ -0.517, 59.194, 87434.028],
        'ac': [ -11.66, 64.927, 396.571, 0.064, 193951.027]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img148.fz': {
        'os': [ -0.341, 59.51, 87419.442],
        'ac': [ -10.466, 64.838, 398.367, 0.066, 193970.822]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img145.fz': {
        'os': [ 0.01, 59.333, 87386.978],
        'ac': [ -11.263, 64.719, 397.453, 0.066, 195231.348]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img142.fz': {
        'os': [ -0.483, 59.68, 87833.176],
        'ac': [ -11.355, 65.077, 399.757, 0.071, 193698.868],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img139.fz': {
        'os': [ -0.005, 59.279, 87535.592],
        'ac': [ -11.301, 64.93, 397.061, 0.074, 194410.147],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img136.fz': {
        'os': [ -0.266, 59.745, 87840.56 ],
        'ac': [ -11.566, 65.156, 399.143, 0.076, 193081.738],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img133.fz': {
        'os': [ -0.164, 59.665, 87822.324],
        'ac': [ -11.092, 65.334, 411.001, 0.078, 193198.059],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img130.fz': {
        'os': [ -0.481, 59.385, 87444.874],
        'ac': [ -10.562, 65.021, 399.202, 0.081, 194859.854],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img127.fz': {
        'os': [ -0.294, 59.192, 87481.826],
        'ac': [ -11.343, 65.142, 400.385, 0.085, 194955.314],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img124.fz': {
        'os': [ -0.425, 59.721, 87232.754],
        'ac': [ -11.132, 65.325, 400.733, 0.085, 194298.829],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img121.fz': {
        'os': [ -0.392, 59.651, 87438.021],
        'ac': [ -10.755, 65.488, 400.629, 0.082, 194875.764],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img118.fz': {
        'os': [ -0.51, 59.641, 87376.276],
        'ac': [ -11.368, 65.436, 401.484, 0.083, 194846.273]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img115.fz': {
        'os': [ -0.252, 60.584, 87743.606],
        'ac': [ -11.068, 66.005, 403.335, 0.088, 194146.206],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img112.fz': {
        'os': [ -0.273, 59.058, 87484.541],
        'ac': [ -10.895, 64.738, 401.384, 0.082, 194120.448],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img109.fz': {
        'os': [ -0.253, 59.093, 87497.08 ],
        'ac': [ -11.507, 64.761, 399.258, 0.085, 194482.02 ]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img106.fz': {
        'os': [ -0.129, 59.317, 87671.988],
        'ac': [ -11.108, 65.126, 396.289, 0.087, 193857.804]
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img103.fz': {
        'os': [ 0.052, 58.997, 87437.662],
        'ac': [ -11.121, 64.922, 398.255, 0.083, 194618.358],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img100.fz': {
        'os': [ -0.293, 59.247, 87407.631],
        'ac': [ -11.191, 64.817, 396.737, 0.085, 195086.393],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img97.fz': {
        'os': [ -0.045, 59.898, 87534.062],
        'ac': [ -11.116, 65.605, 399.416, 0.085, 195100.49 ],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img94.fz': {
        'os': [ -0.297, 60.813, 87404.554],
        'ac': [ -10.749, 66.788, 401.777, 0.087, 194628.093],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img91.fz': {
        'os': [ -0.231, 62.059, 87401.257],
        'ac': [ -11.019, 67.621, 398.842, 0.086, 195249.606],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img88.fz': {
        'os': [ -0.306, 61.009, 87479.733],
        'ac': [ -10.94, 66.654, 400.212, 0.086, 195821.729],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img85.fz': {
        'os': [ -0.249, 58.969, 87408.985],
        'ac': [ -10.828, 64.984, 397.614, 0.088, 195106.664],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img82.fz': {
        'os': [ -0.154, 59.085, 87438.336],
        'ac': [ -11.052, 64.934, 399.838, 0.092, 194125.971],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img79.fz': {
        'os': [ -0.13, 59.105, 87457.792],
        'ac': [ -10.677, 64.644, 399.079, 0.09, 195090.389],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img76.fz': {
        'os': [ -0.11, 58.956, 87519.72 ],
        'ac': [ -10.96, 64.889, 395.746, 0.095, 195046.987],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img73.fz': {
        'os': [ -0.005, 58.951, 87522.034],
        'ac': [ -11.065, 64.947, 401.609, 0.095, 195250.731],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img70.fz': {
        'os': [ -0.198, 59.872, 87862.163],
        'ac': [ 386.326, 67.585, 402.993, 0.09, 19290.243],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img67.fz': {
        'os': [ -0.177, 59.333, 87423.912],
        'ac': [ -10.628, 64.997, 399.421, 0.105, 194309.604],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img64.fz': {
        'os': [ -0.264, 59.341, 87328.298],
        'ac': [ -11.537, 64.684, 398.197, 0.111, 194660.964],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img61.fz': {
        'os': [-0.4726600018148671, 59.18113278533696, 87738.71971959743],
        'oserr': [0.16740236047590382, 0.1206823452548757, 246.8012099390647],
        'ac': [-11.533478287448776, 64.90184019883736, 397.6652534661094, 0.11543207342016622, 196019.1639705012],
        'acerr': [0.23536143653333386, 0.16607367015541696, 0.6844082490938922, 0.001196172096767872, 653.271837003891],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img58.fz': {
        'os': [-0.18547200663037625, 59.16317844409124, 87819.15634263899],
        'oserr': [0.10638863549226993, 0.07647328329892758, 157.0105638236852],
        'ac': [-10.927313173010509, 64.71523165038761, 402.2284111730161, 0.12345020021681957, 195000.86460171643],
        'acerr': [0.1684770106501128, 0.11854953831327351, 0.45776491807455744, 0.0008609413335827681, 465.1018382947694],
    },    
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img55.fz': {
        'os': [-0.06331844579807741, 59.24787736806688, 87870.26418164143],
        'oserr': [0.11891113856071252, 0.08617691918643021, 175.3639092008104],
        'ac': [-11.03477706157318, 65.1571976337272, 399.55540983445485, 0.1300156377146666, 195082.64610743022],
        'acerr': [0.19681610135472316, 0.13839625363607078, 0.516789466635856, 0.0010226501604546688, 537.4544421287626],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img52.fz': {
        'os': [-0.0399197206277865, 59.198246967045755, 88081.57890551595],
        'oserr': [0.05562955368291622, 0.040311400442808974, 82.33390556184544],
        'ac': [-11.017430250593142, 65.3638304789849, 401.3787330260013, 0.1407613278574609, 194847.72108836245],
        'acerr': [0.12450702671913144, 0.0870208585175435, 0.3171147800386809, 0.0006713056415615399, 336.92524767300966],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img49.fz': {
        'os': [-0.3217205869386259, 59.520455903002635, 88216.17873310857],
        'oserr': [0.048587414296920685, 0.03502233752613649, 71.62693601170714],
        'ac': [-11.193921184239299, 65.44381990332606, 402.3302799865032, 0.153921894034195, 194279.16606747746],
        'acerr': [0.13616462008226293, 0.09494178916263736, 0.3325960825625976, 0.0007656310948385075, 364.9581818018499],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img46.fz': {
        'os': [-0.07244793071000925, 59.48094450874807, 88026.96183642559],
        'oserr': [0.06278562955238533, 0.04545882758039976, 92.41376346688602],
        'ac': [387.80196816057787, 65.77674345602891, 401.48402080938973, 0.13267176604220224, 30146.42076623923],
        'acerr': [2.37659887724815, 1.6531757664426379, 6.394739490713986, 0.012652648666339354, 1004.2506624480876],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img43.fz': {
        'os': [-0.27333637708788183, 59.20447178797045, 87816.35360133021],
        'oserr': [0.07448664376443001, 0.05370455738283573, 109.74885649802565],
        'ac': [-11.322670024361152, 65.09491527926251, 403.04165246580646, 0.18390420862604503, 194340.81057563383],
        'acerr': [0.13972057115183406, 0.09580372442245536, 0.30519972308959853, 0.000852806991492934, 370.4549277101024],
    },
    '/share/storage2/connie/data/runs/101/runID101_skp8888_NSAMP400_EXP0_img40.fz': {
        'os': [-0.3356041516049653, 59.535595711554954, 88153.79879023216],
        'oserr': [0.0530297662166777, 0.0384689899722141, 78.13000623907769],
        'ac': [390.3941918379343, 67.55176723014534, 407.9911853215681, 0.16515512014255562, 36330.944169339695],
        'acerr': [2.212450725437896, 1.5157124164699232, 5.193908616398216, 0.012690262672911977, 1081.483489618588],
    },    
}

getcol = lambda s, c: np.array( 
    [ (
        int(re.search("img([0-9]+)\.fz", path).groups()[0]), 
        data[s][c] if s in data else 0, 
    )
    for path, data in evo.items() ] 
)

runID = getcol('os', 1)[:,0]
fig, axes = plt.subplots(2)
axes[0].plot( runID, getcol('os', 1)[:,1], '.', label = 'os noise' )
axes[0].plot( runID, getcol('ac', 1)[:,1], '.', label = 'ac noise' )
axes[0].set_ylabel(r'$\sigma$ [e-]')
axes[0].grid(True)
axes[0].legend(frameon=False)
for range_, T in temperatures.items():
    mask = ( runID > range_[0] if range_[0] is not None else True ) & ( runID <= range_[1] if range_[1] is not None else True)
    axes[1].errorbar( 
        runID[mask], 
        getcol('ac', 3)[:,1][mask] * 24/1.5, 
#         yerr = getcol('acerr', 3)[:,1][mask] * 24/1.5, 
        fmt=".", 
        ms=5, 
        label = T
    )
axes[1].set_ylabel(r'$\lambda$ [e-/pix/day]')
axes[1].set_yscale('log')
axes[1].legend(title="dark current", frameon=False)
axes[1].grid(True)
plt.draw()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …